In [40]:
import sqlite3
import geopandas as gpd
import pandas as pd

In [41]:
def latlonTable2GDF(table: pd.DataFrame, colName_lat: str, colName_lon: str):
    return gpd.GeoDataFrame(table, geometry=gpd.points_from_xy(table[colName_lon], table[colName_lat])).set_crs(6668)

In [5]:
# 対象地区（横浜市）
target_jcodes = [
    "14101",
    "14102",
    "14103",
    "14104",
    "14105",
    "14106",
    "14107",
    "14108",
    "14109",
    "14110",
    "14111",
    "14112",
    "14113",
    "14114",
    "14115",
    "14116",
    "14117",
    "14118"
]

In [75]:
# target_jcodesをカンマ区切りの文字列に変換
# SQL IN 句では、リストのように扱える
placeholders = ','.join('?' for _ in target_jcodes)

# データベース接続
conn = sqlite3.connect("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\2015_1_vJHPCN2019.sqlite3")
cursor = conn.cursor()
# SQL クエリを実行して town_code が target_jcodes に含まれるレコードを抽出
query = f"""
WITH filtered_household AS (
    SELECT *
    FROM household
    WHERE SUBSTR(town_code, 1, 5) IN ({placeholders})
), filtered_person AS (
    SELECT
        person.age,
        person.sex_id,
        person.role_household_type_id,
        building.latitude,
        building.longitude,
        worker.employment_type_id,
        worker.company_id,
        person.household_id
    FROM
        person
    JOIN 
        filtered_household ON person.household_id = filtered_household.household_id
    JOIN 
        building ON filtered_household.building_id = building.building_id
    LEFT JOIN 
        worker ON person.person_id = worker.person_id
)

SELECT
    filtered_person.age,
    filtered_person.sex_id,
    filtered_person.role_household_type_id,
    filtered_person.latitude,
    filtered_person.longitude,
    filtered_person.employment_type_id,
    filtered_person.company_id,
    company.industry_type_id,
    filtered_person.household_id
FROM 
    filtered_person
LEFT JOIN 
    company ON filtered_person.company_id = company.company_id
"""

cursor.execute(query, target_jcodes)
result = cursor.fetchall()
conn.close()

In [76]:
len(list(result))

3495691

In [77]:
yokohama = pd.DataFrame(result,columns=["age","sex_id","role_household_type_id","latitude","longitude","employment_type_id","company_id","industry_type_id","household_id"])

In [78]:
# yokohama.to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\sp_data_14100.csv")
yokohama = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\sp_data_14100.csv")

対象範囲にclip

In [79]:
yokohama

,age,sex_id,role_household_type_id,latitude,longitude,employment_type_id,company_id,industry_type_id,household_id
0,15,0,0,35.536595,139.657303,NaN,NaN,NaN,19438213
1,15,0,0,35.492273,139.661276,NaN,NaN,NaN,19438214
2,15,0,0,35.529866,139.670720,NaN,NaN,NaN,19438215
3,15,0,0,35.519671,139.682110,NaN,NaN,NaN,19438216
4,15,0,0,35.503321,139.672198,NaN,NaN,NaN,19438217
...,...,...,...,...,...,...,...,...,...
3495686,26,0,20,35.527540,139.587707,10.0,63.0,180.0,21007197
3495687,21,1,21,35.527540,139.587707,NaN,NaN,NaN,21007197
3495688,26,0,20,35.527540,139.587707,10.0,13.0,50.0,21007197
3495689,30,0,20,35.527540,139.587707,NaN,NaN,NaN,21007197


In [80]:
mesh_poly = gpd.read_file("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\mesh\\mesh_geom.shp")

In [81]:
sjoined = latlonTable2GDF(yokohama,"latitude","longitude").sjoin(mesh_poly.to_crs(6668),how="left",predicate="intersects").filter(
    items=["age","sex_id","role_household_type_id","employment_type_id","industry_type_id","latitude","longitude",
    "household_id","mesh_code"])

In [82]:
sjoined.query("mesh_code == mesh_code").to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\pop\\spdata.csv",index=False)

In [83]:
sjoined

,age,sex_id,role_household_type_id,employment_type_id,industry_type_id,latitude,longitude,household_id,mesh_code
0,15,0,0,NaN,NaN,35.536595,139.657303,19438213,NaN
1,15,0,0,NaN,NaN,35.492273,139.661276,19438214,NaN
2,15,0,0,NaN,NaN,35.529866,139.670720,19438215,NaN
3,15,0,0,NaN,NaN,35.519671,139.682110,19438216,NaN
4,15,0,0,NaN,NaN,35.503321,139.672198,19438217,NaN
...,...,...,...,...,...,...,...,...,...
3495686,26,0,20,10.0,180.0,35.527540,139.587707,21007197,NaN
3495687,21,1,21,NaN,NaN,35.527540,139.587707,21007197,NaN
3495688,26,0,20,10.0,50.0,35.527540,139.587707,21007197,NaN
3495689,30,0,20,NaN,NaN,35.527540,139.587707,21007197,NaN


模擬個票人口の調整

In [84]:
target = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\pop\\spdata.csv")
target

,age,sex_id,role_household_type_id,employment_type_id,industry_type_id,latitude,longitude,household_id,mesh_code
0,15,0,0,NaN,NaN,35.412949,139.583328,19794381,5339049641
1,15,0,0,NaN,NaN,35.425309,139.604004,19794383,5339141812
2,15,0,0,NaN,NaN,35.428369,139.578979,19794386,5339141614
3,15,0,0,NaN,NaN,35.423831,139.601588,19794388,5339140833
4,15,0,0,NaN,NaN,35.424840,139.604145,19794390,5339140834
...,...,...,...,...,...,...,...,...,...
225754,24,0,20,10.0,90.0,35.405697,139.592515,20482501,5339048732
225755,14,1,21,NaN,NaN,35.405697,139.592515,20482501,5339048732
225756,26,1,21,NaN,NaN,35.405697,139.592515,20482501,5339048732
225757,19,0,20,NaN,NaN,35.405697,139.592515,20482501,5339048732


In [93]:
n = 1
target.sample(n=n).to_csv(f"C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\pop\\spdata_{n}.csv",index=False)

In [89]:
target

,age,sex_id,role_household_type_id,employment_type_id,industry_type_id,latitude,longitude,household_id,mesh_code
0,15,0,0,NaN,NaN,35.412949,139.583328,19794381,5339049641
1,15,0,0,NaN,NaN,35.425309,139.604004,19794383,5339141812
2,15,0,0,NaN,NaN,35.428369,139.578979,19794386,5339141614
3,15,0,0,NaN,NaN,35.423831,139.601588,19794388,5339140833
4,15,0,0,NaN,NaN,35.424840,139.604145,19794390,5339140834
...,...,...,...,...,...,...,...,...,...
225754,24,0,20,10.0,90.0,35.405697,139.592515,20482501,5339048732
225755,14,1,21,NaN,NaN,35.405697,139.592515,20482501,5339048732
225756,26,1,21,NaN,NaN,35.405697,139.592515,20482501,5339048732
225757,19,0,20,NaN,NaN,35.405697,139.592515,20482501,5339048732
